In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from typing import List, Optional
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from src.utils.file_utils import convert_time_to_seconds, create_full_date

In [3]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM results", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [104]:
conn.close()

In [98]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM athletes", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [99]:
df

,seq,name,club,sex
0,4851475245544851495050494554,VAUDOIS Aurelien,- Athle 92*,M
1,495046534653504950494257,DIRAT Franck,- Athle 92*,M
2,49504851445551484851475246535148,POTTIER Quentin,- Athle 92*,M


In [27]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
db_url = os.getenv("DB_URL")

# Crée l'engine SQLAlchemy
engine = create_engine(db_url)

# Exemple : lire la table results
df = pd.read_sql_query("SELECT * FROM athletes", engine)
print(df.head())

                            seq              name          club sex  \
0  4851475245544851495050494554  VAUDOIS Aurelien   - Athle 92*   M   

                 last_update  
0 2025-04-20 23:22:54.803567  
